### using static (non-trainable) glove vectors with convolutional layers to classify newsgroup_20 documents

https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html


In [15]:
import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, Dropout, Embedding, GlobalMaxPooling1D, MaxPooling1D
from keras.models import Model,Sequential

from helpers import (files_helper, 
                     metrics_helper, 
                     tags_helper,
                     texts_helper)


In [2]:
SEED=np.random.randint(1,1000)
SEED

672

In [3]:
np.random.seed(SEED)

In [4]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM=100
LABELS_MIN_DOC_COUNT = int(10)
BATCH_SIZE=32
NUM_EPOCHS=10
TOKENIZER_FILTERS='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n' # default keras filters

In [6]:
embeddings_index = files_helper.read_glove_wiki(d=EMBEDDING_DIM)

In [7]:
texts,labels_index,labels_0 = files_helper.read_20_newgroup_files()       

In [8]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS,
                      filters=TOKENIZER_FILTERS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
inverse_word_index = texts_helper.build_inverse_word_index(word_index)

In [9]:
len(word_index),len(labels_index)

(174074, 20)

In [10]:
# [inverse_word_index[idx] for idx in sequences[0]]

In [11]:
data = pad_sequences(sequences,maxlen=MAX_SEQUENCE_LENGTH)

labels_1 = to_categorical(np.asarray(labels_0))

indices = np.arange(data.shape[0])
np.random.shuffle(indices)

data = data[indices]
labels_2 = labels_1[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

X_train = data[:-num_validation_samples]
y_train = labels_2[:-num_validation_samples]
X_val = data[-num_validation_samples:]
y_val = labels_2[-num_validation_samples:]

data.shape,labels_2.shape

((19997, 1000), (19997, 20))

In [12]:
embedding_matrix = np.zeros((len(word_index)+1,EMBEDDING_DIM))

for word,i in word_index.items():
    
    if i >= MAX_NB_WORDS:
        continue
    
    embedding_vector = embeddings_index.get(word)
    
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [13]:
embedding_matrix.shape

(174075, 100)

In [16]:
model = Sequential()

model.add(Embedding(len(word_index)+1,
                           EMBEDDING_DIM,
                           weights=[embedding_matrix],
                           input_length=MAX_SEQUENCE_LENGTH,
                           trainable = False))

model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(128,activation='relu'))
model.add(Dense(len(labels_index),activation='softmax'))
model.compile(loss ='categorical_crossentropy',
             optimizer='rmsprop',
             metrics=['acc'])

In [17]:
%%time

model.fit(X_train,y_train, validation_data=(X_val, y_val),
         epochs=NUM_EPOCHS, batch_size=BATCH_SIZE)

Train on 15998 samples, validate on 3999 samples
Epoch 1/10
15998/15998 [==============================] - 19s - loss: 2.2256 - acc: 0.2180 - val_loss: 1.7256 - val_acc: 0.3768.
Epoch 2/10
15998/15998 [==============================] - 18s - loss: 1.3845 - acc: 0.5092 - val_loss: 1.1613 - val_acc: 0.5934
Epoch 3/10
15998/15998 [==============================] - 18s - loss: 1.0131 - acc: 0.6485 - val_loss: 0.9806 - val_acc: 0.6599
Epoch 4/10
15998/15998 [==============================] - 19s - loss: 0.8084 - acc: 0.7200 - val_loss: 0.9108 - val_acc: 0.7039
Epoch 5/10
15998/15998 [==============================] - 18s - loss: 0.6563 - acc: 0.7789 - val_loss: 0.9471 - val_acc: 0.7007
Epoch 6/10
15998/15998 [==============================] - 18s - loss: 0.5245 - acc: 0.8210 - val_loss: 0.9683 - val_acc: 0.7149
Epoch 7/10
15998/15998 [==============================] - 17s - loss: 0.4386 - acc: 0.8532 - val_loss: 0.9016 - val_acc: 0.7439
Epoch 8/10
15998/15998 [==============================